In [1]:
#help("modules")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, SGD,Adamax
from sklearn.preprocessing import StandardScaler
import scipy.io as sio
from pathlib import Path
import pandas as pd
import tensorflow as tf
import os
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#import winsound
from keras.regularizers import l2, l1_l2, l1
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import load_model

import matplotlib.pyplot as plt


In [2]:
def load_data():
    return sio.loadmat('data_elastic_200_khz_wave_lim_5_narrow_angle_3_10k_test.mat')

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Load the data
data = load_data()
(data_i2, data_o2) = (data['data_inp'], data['data_out'])

def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))

    return (1 - SS_res / (SS_tot + K.epsilon()))

X_test=data_i2
Y_test=data_o2

In [3]:
def load_data():
    return sio.loadmat('data_elastic_200_khz_wave_lim_5_narrow_angle_3_100k.mat')

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Load the data
data = load_data()
(data_i1, data_o1) = (data['data_inp'], data['data_out'])

def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))

    return (1 - SS_res / (SS_tot + K.epsilon()))


X_train = data_i1 
Y_train = data_o1

In [4]:
noise_level=0.03 #1 percent randomly distributed noise

random_matrix_train=(np.random.random_sample((np.size(X_train,0), np.size(X_train,1)))-0.5)*2 

X_train_noise = X_train*(random_matrix_train*noise_level)+X_train


In [5]:
random_matrix_test=(np.random.random_sample((np.size(X_test,0), np.size(X_test,1)))-0.5)*2 

X_test_noise = X_test*(random_matrix_test*noise_level)+X_test

In [6]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

os.environ["CUDA_VISIBLE_DEVICES"]="1"
def model_builder(hp):
    # Set the input shape
    feature_vector_length = data_i1.shape[1]
    result_vector_length = data_o1.shape[1]
    input_len = (feature_vector_length,)
    
    model = Sequential()
    model.add(tf.keras.layers.LayerNormalization(
        axis=-1,
        epsilon=0.001,
        center=True,
        scale=True,
        beta_initializer="zeros",
        gamma_initializer="ones",
        beta_regularizer=None,
        gamma_regularizer=None,
        beta_constraint=None,
        gamma_constraint=None))
    model.add(Dense(100, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=l1_l2(l1=1e-5, l2=1e-4), activity_regularizer=l1_l2(l1=1e-5, l2=1e-4), input_shape=input_len, activation='relu'))
    
    hp_units1 = hp.Int('units1', min_value=100, max_value=5000, step=50)
    model.add(keras.layers.Dense(units=hp_units1, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=l1_l2(l1=1e-5, l2=1e-4), activity_regularizer=l1_l2(l1=1e-5, l2=1e-4), input_shape=input_len, activation='relu'))
    hp_units2 = hp.Int('units2', min_value=100, max_value=5000, step=50)
    model.add(keras.layers.Dense(units=hp_units2, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=l1_l2(l1=1e-5, l2=1e-4), activity_regularizer=l1_l2(l1=1e-5, l2=1e-4), input_shape=input_len, activation='relu'))
    
    model.add(keras.layers.Dense(result_vector_length, activation=None))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3, decay=1e-3 / np.size(X_train,0)),
                loss="mape",
                metrics=['mae', 'mape', r_square])
    return model

es = EarlyStopping(monitor='val_mape', mode='min', verbose=1, patience=50)

tuner = kt.Hyperband(model_builder,
                     objective='val_mape',
                     hyperband_iterations=1,
                     max_epochs=100,
                     factor=3,
                     overwrite=True)

tuner.search(X_train_noise, Y_train, epochs=100, batch_size=256, verbose=2, validation_split=0.1, callbacks=[es])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

#print(f"""
#The hyperparameter search is complete. The optimal number of units in the first densely-connected
#layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
#is {best_hps.get('learning_rate')}.
#""")


Trial 254 Complete [00h 07m 02s]
val_mape: 3.4618027210235596

Best val_mape So Far: 3.37614369392395
Total elapsed time: 03h 35m 11s
INFO:tensorflow:Oracle triggered exit


In [7]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history_model = model.fit(X_train_noise, Y_train, epochs=1000, batch_size=256, verbose=2, validation_split=0.1, callbacks=[es])


Epoch 1/1000
352/352 - 5s - loss: 16.3652 - mae: 0.8950 - mape: 16.0097 - r_square: 0.7806 - val_loss: 11.5276 - val_mae: 0.5135 - val_mape: 11.2037 - val_r_square: 0.9510 - 5s/epoch - 13ms/step
Epoch 2/1000
352/352 - 4s - loss: 10.0659 - mae: 0.4173 - mape: 9.7501 - r_square: 0.9677 - val_loss: 9.4426 - val_mae: 0.3718 - val_mape: 9.1349 - val_r_square: 0.9762 - 4s/epoch - 11ms/step
Epoch 3/1000
352/352 - 4s - loss: 8.6137 - mae: 0.3182 - mape: 8.3129 - r_square: 0.9828 - val_loss: 8.2650 - val_mae: 0.2822 - val_mape: 7.9705 - val_r_square: 0.9874 - 4s/epoch - 11ms/step
Epoch 4/1000
352/352 - 4s - loss: 7.7682 - mae: 0.2739 - mape: 7.4796 - r_square: 0.9880 - val_loss: 8.7510 - val_mae: 0.3207 - val_mape: 8.4690 - val_r_square: 0.9841 - 4s/epoch - 11ms/step
Epoch 5/1000
352/352 - 4s - loss: 7.2547 - mae: 0.2509 - mape: 6.9749 - r_square: 0.9902 - val_loss: 7.2231 - val_mae: 0.2594 - val_mape: 6.9454 - val_r_square: 0.9888 - 4s/epoch - 11ms/step
Epoch 6/1000
352/352 - 4s - loss: 7.0362

Epoch 44/1000
352/352 - 4s - loss: 4.2583 - mae: 0.1393 - mape: 4.0511 - r_square: 0.9971 - val_loss: 4.2510 - val_mae: 0.1338 - val_mape: 4.0442 - val_r_square: 0.9974 - 4s/epoch - 11ms/step
Epoch 45/1000
352/352 - 4s - loss: 4.2650 - mae: 0.1408 - mape: 4.0580 - r_square: 0.9971 - val_loss: 4.5417 - val_mae: 0.1660 - val_mape: 4.3351 - val_r_square: 0.9956 - 4s/epoch - 11ms/step
Epoch 46/1000
352/352 - 4s - loss: 4.2243 - mae: 0.1396 - mape: 4.0178 - r_square: 0.9971 - val_loss: 4.3064 - val_mae: 0.1477 - val_mape: 4.0998 - val_r_square: 0.9968 - 4s/epoch - 11ms/step
Epoch 47/1000
352/352 - 4s - loss: 4.2352 - mae: 0.1405 - mape: 4.0281 - r_square: 0.9971 - val_loss: 4.2520 - val_mae: 0.1362 - val_mape: 4.0462 - val_r_square: 0.9974 - 4s/epoch - 11ms/step
Epoch 48/1000
352/352 - 4s - loss: 4.1347 - mae: 0.1352 - mape: 3.9273 - r_square: 0.9973 - val_loss: 4.3654 - val_mae: 0.1532 - val_mape: 4.1583 - val_r_square: 0.9965 - 4s/epoch - 11ms/step
Epoch 49/1000
352/352 - 4s - loss: 4.188

Epoch 87/1000
352/352 - 4s - loss: 3.5951 - mae: 0.1156 - mape: 3.3762 - r_square: 0.9980 - val_loss: 3.8067 - val_mae: 0.1198 - val_mape: 3.5865 - val_r_square: 0.9980 - 4s/epoch - 11ms/step
Epoch 88/1000
352/352 - 4s - loss: 3.6123 - mae: 0.1166 - mape: 3.3930 - r_square: 0.9980 - val_loss: 3.9816 - val_mae: 0.1196 - val_mape: 3.7631 - val_r_square: 0.9981 - 4s/epoch - 11ms/step
Epoch 89/1000
352/352 - 4s - loss: 3.6127 - mae: 0.1163 - mape: 3.3940 - r_square: 0.9980 - val_loss: 3.9529 - val_mae: 0.1256 - val_mape: 3.7343 - val_r_square: 0.9977 - 4s/epoch - 11ms/step
Epoch 90/1000
352/352 - 4s - loss: 3.5994 - mae: 0.1158 - mape: 3.3798 - r_square: 0.9980 - val_loss: 4.0589 - val_mae: 0.1310 - val_mape: 3.8387 - val_r_square: 0.9977 - 4s/epoch - 11ms/step
Epoch 91/1000
352/352 - 4s - loss: 3.5743 - mae: 0.1140 - mape: 3.3545 - r_square: 0.9981 - val_loss: 3.6816 - val_mae: 0.1146 - val_mape: 3.4629 - val_r_square: 0.9981 - 4s/epoch - 11ms/step
Epoch 92/1000
352/352 - 4s - loss: 3.611

Epoch 130/1000
352/352 - 4s - loss: 3.3192 - mae: 0.1054 - mape: 3.0855 - r_square: 0.9984 - val_loss: 3.5526 - val_mae: 0.1093 - val_mape: 3.3186 - val_r_square: 0.9983 - 4s/epoch - 11ms/step
Epoch 131/1000
352/352 - 4s - loss: 3.2932 - mae: 0.1037 - mape: 3.0586 - r_square: 0.9984 - val_loss: 4.0052 - val_mae: 0.1307 - val_mape: 3.7704 - val_r_square: 0.9976 - 4s/epoch - 11ms/step
Epoch 132/1000
352/352 - 4s - loss: 3.3392 - mae: 0.1057 - mape: 3.1048 - r_square: 0.9984 - val_loss: 3.6229 - val_mae: 0.1191 - val_mape: 3.3875 - val_r_square: 0.9979 - 4s/epoch - 11ms/step
Epoch 133/1000
352/352 - 4s - loss: 3.3163 - mae: 0.1040 - mape: 3.0813 - r_square: 0.9985 - val_loss: 3.9353 - val_mae: 0.1423 - val_mape: 3.7004 - val_r_square: 0.9969 - 4s/epoch - 11ms/step
Epoch 134/1000
352/352 - 4s - loss: 3.2792 - mae: 0.1031 - mape: 3.0440 - r_square: 0.9985 - val_loss: 3.6555 - val_mae: 0.1141 - val_mape: 3.4196 - val_r_square: 0.9981 - 4s/epoch - 11ms/step
Epoch 135/1000
352/352 - 4s - loss:

Epoch 173/1000
352/352 - 4s - loss: 3.1409 - mae: 0.0984 - mape: 2.8928 - r_square: 0.9986 - val_loss: 3.4276 - val_mae: 0.1055 - val_mape: 3.1793 - val_r_square: 0.9984 - 4s/epoch - 11ms/step
Epoch 174/1000
352/352 - 4s - loss: 3.1425 - mae: 0.0980 - mape: 2.8936 - r_square: 0.9986 - val_loss: 3.4147 - val_mae: 0.1061 - val_mape: 3.1661 - val_r_square: 0.9984 - 4s/epoch - 11ms/step
Epoch 175/1000
352/352 - 4s - loss: 3.1350 - mae: 0.0967 - mape: 2.8854 - r_square: 0.9987 - val_loss: 3.7021 - val_mae: 0.1143 - val_mape: 3.4523 - val_r_square: 0.9983 - 4s/epoch - 11ms/step
Epoch 176/1000
352/352 - 4s - loss: 3.1183 - mae: 0.0970 - mape: 2.8676 - r_square: 0.9987 - val_loss: 3.4362 - val_mae: 0.1106 - val_mape: 3.1854 - val_r_square: 0.9982 - 4s/epoch - 11ms/step
Epoch 177/1000
352/352 - 4s - loss: 3.0925 - mae: 0.0955 - mape: 2.8423 - r_square: 0.9987 - val_loss: 3.4850 - val_mae: 0.1049 - val_mape: 3.2342 - val_r_square: 0.9985 - 4s/epoch - 11ms/step
Epoch 178/1000
352/352 - 4s - loss:

Epoch 216/1000
352/352 - 4s - loss: 2.9791 - mae: 0.0914 - mape: 2.7181 - r_square: 0.9988 - val_loss: 3.4647 - val_mae: 0.1050 - val_mape: 3.2038 - val_r_square: 0.9985 - 4s/epoch - 11ms/step
Epoch 217/1000
352/352 - 4s - loss: 2.9925 - mae: 0.0914 - mape: 2.7321 - r_square: 0.9988 - val_loss: 3.4653 - val_mae: 0.1033 - val_mape: 3.2047 - val_r_square: 0.9986 - 4s/epoch - 11ms/step
Epoch 218/1000
352/352 - 4s - loss: 3.0100 - mae: 0.0924 - mape: 2.7487 - r_square: 0.9988 - val_loss: 3.4152 - val_mae: 0.1012 - val_mape: 3.1531 - val_r_square: 0.9986 - 4s/epoch - 11ms/step
Epoch 219/1000
352/352 - 4s - loss: 2.9977 - mae: 0.0921 - mape: 2.7356 - r_square: 0.9988 - val_loss: 3.8638 - val_mae: 0.1216 - val_mape: 3.6017 - val_r_square: 0.9981 - 4s/epoch - 11ms/step
Epoch 220/1000
352/352 - 4s - loss: 2.9999 - mae: 0.0923 - mape: 2.7381 - r_square: 0.9988 - val_loss: 3.3625 - val_mae: 0.1006 - val_mape: 3.1007 - val_r_square: 0.9986 - 4s/epoch - 11ms/step
Epoch 221/1000
352/352 - 4s - loss:

Epoch 259/1000
352/352 - 4s - loss: 2.8892 - mae: 0.0882 - mape: 2.6172 - r_square: 0.9989 - val_loss: 3.3511 - val_mae: 0.1009 - val_mape: 3.0799 - val_r_square: 0.9986 - 4s/epoch - 11ms/step
Epoch 260/1000
352/352 - 4s - loss: 2.9097 - mae: 0.0887 - mape: 2.6387 - r_square: 0.9989 - val_loss: 3.8017 - val_mae: 0.1328 - val_mape: 3.5300 - val_r_square: 0.9974 - 4s/epoch - 11ms/step
Epoch 261/1000
352/352 - 4s - loss: 2.9030 - mae: 0.0879 - mape: 2.6302 - r_square: 0.9989 - val_loss: 3.4541 - val_mae: 0.1084 - val_mape: 3.1808 - val_r_square: 0.9984 - 4s/epoch - 11ms/step
Epoch 262/1000
352/352 - 4s - loss: 2.8708 - mae: 0.0869 - mape: 2.5984 - r_square: 0.9989 - val_loss: 3.4082 - val_mae: 0.0988 - val_mape: 3.1364 - val_r_square: 0.9987 - 4s/epoch - 11ms/step
Epoch 263/1000
352/352 - 4s - loss: 2.9005 - mae: 0.0881 - mape: 2.6278 - r_square: 0.9989 - val_loss: 3.3306 - val_mae: 0.0982 - val_mape: 3.0574 - val_r_square: 0.9987 - 4s/epoch - 11ms/step
Epoch 264/1000
352/352 - 4s - loss:

Epoch 302/1000
352/352 - 4s - loss: 2.8095 - mae: 0.0841 - mape: 2.5283 - r_square: 0.9990 - val_loss: 3.2772 - val_mae: 0.0988 - val_mape: 2.9958 - val_r_square: 0.9987 - 4s/epoch - 11ms/step
Epoch 303/1000
352/352 - 4s - loss: 2.8289 - mae: 0.0854 - mape: 2.5470 - r_square: 0.9990 - val_loss: 3.4731 - val_mae: 0.1050 - val_mape: 3.1905 - val_r_square: 0.9985 - 4s/epoch - 11ms/step
Epoch 304/1000
352/352 - 4s - loss: 2.8260 - mae: 0.0854 - mape: 2.5439 - r_square: 0.9990 - val_loss: 3.3886 - val_mae: 0.1054 - val_mape: 3.1067 - val_r_square: 0.9985 - 4s/epoch - 11ms/step
Epoch 305/1000
352/352 - 4s - loss: 2.8373 - mae: 0.0856 - mape: 2.5547 - r_square: 0.9990 - val_loss: 3.4007 - val_mae: 0.1016 - val_mape: 3.1181 - val_r_square: 0.9986 - 4s/epoch - 11ms/step
Epoch 306/1000
352/352 - 4s - loss: 2.8139 - mae: 0.0849 - mape: 2.5322 - r_square: 0.9990 - val_loss: 3.3729 - val_mae: 0.1024 - val_mape: 3.0910 - val_r_square: 0.9986 - 4s/epoch - 11ms/step
Epoch 307/1000
352/352 - 4s - loss:

In [8]:
val_acc_per_epoch = history_model.history['mape']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 313


In [9]:
predict_noise=model.predict(X_test_noise)


In [10]:
print(np.mean(np.abs(Y_test[:,0]-predict_noise[:,0])/(Y_test[:,0]))*100)
print(np.mean(np.abs(Y_test[:,1]-predict_noise[:,1])/(Y_test[:,1]))*100)
print(np.mean(np.abs(Y_test[:,2]-predict_noise[:,2])/(Y_test[:,2]))*100)
print(np.mean(np.abs(Y_test[:,3]-predict_noise[:,3])/(Y_test[:,3]))*100)
print(np.mean(np.abs(Y_test[:,4]-predict_noise[:,4])/(Y_test[:,4]))*100)
print(np.mean(np.abs(Y_test[:,5]-predict_noise[:,5])/(Y_test[:,5]))*100)
print(np.mean(np.abs(Y_test[:,6]-predict_noise[:,6])/(Y_test[:,6]))*100)
print(np.mean(np.abs(Y_test[:,7]-predict_noise[:,7])/(Y_test[:,7]))*100)
print(np.mean(np.abs(Y_test[:,8]-predict_noise[:,8])/(Y_test[:,8]))*100)

print(np.size(X_test_noise,0))
print(np.size(X_train_noise,0))

2.5043625091448636
4.815532248271523
6.476135508392292
2.095583434848337
6.06035179122891
2.0143881543825537
0.9929283313541781
1.2479851054960251
1.0929955790597994
10000
100000


In [11]:
model.save('beech_200_khz_200k_model_noise_3_optimized.h5')

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_model.history) 

# save to json:  
hist_json_file = 'beech_200_khz_200k_model_noise_3_optimized_history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

# or save to csv: 
#hist_csv_file = 'history.csv'
#with open(hist_csv_file, mode='w') as f:
#    hist_df.to_csv(f)


In [12]:
print(best_hps.get('units1'))
print(best_hps.get('units2'))

1900
850
